In [0]:
!pip install azureml-sdk

     |████████████████████████████████| 137 kB 1.0 MB/s 
     |████████████▋  

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 4.3 MB 79.3 MB/s 
     |████████████████████████████████| 151 kB 83.7 MB/s 
     |████████████████████████████████| 48 kB 6.5 MB/s 
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10502 sha256=e1902ca04d2e1955789f12e5665f4d24a08469f27f9d9c424080e4240696251a
  Stored in directory: /root/.cache/pip/wheels/d5/60/5d/807081f971b004ab5c86eba746c6b5e72d0258215bf2425d68
Successfully built fusepy
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f94c9aea-fdb5-4bdc-b16a-b769beb83275
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: cryptography
  

In [0]:
from azureml.core import Workspace

In [0]:
!pip install tensorflow==2.5.0

     |▎                               | 3

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 4.9 MB 82.6 MB/s 
     |████████████████████████████████| 781 kB 51.9 MB/s 
     |████████████████████████████████| 242 kB 77.4 MB/s 
     |████████████████████████████████| 182 kB 76.6 MB/s 
     |████████████████████████████████| 181 kB 82.0 MB/s 
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=2cd2b44cdb39b1f209fc7ebb605b7d53ef8b5769b94d7dafa648b4cab7253382
  Stored in directory: /root/.cache/pip/wheels/b6/0d/90/0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
  Created wheel for wrapt: filename=wrapt-1.12.1-cp39-cp39-linux_x86_64.whl size=75955 sha256=64b703ea1c3fb4997ddfd0ad6310cfd6341c927c6ffd3b4db4e598d83e3e5184
  Stored in directory: /root/.cache/pip/wheels/98/23/68/efe259aaca055e93b08e74fbe512819c69a2155c11ba3c0f10
Successfully built termcolor wrapt
  Attempting uninstall: six
    Found exist

In [0]:
{
"subscription_id": "9b5bbfae-d5d1-4aae-a2ca-75159c0c887d",
"resource_group": "cgm-ml-prod-ci-rg",
"workspace_name": "cgm-ml-prod-ci-azml"
}

Out[4]: {'subscription_id': '9b5bbfae-d5d1-4aae-a2ca-75159c0c887d',
 'resource_group': 'cgm-ml-prod-ci-rg',
 'workspace_name': 'cgm-ml-prod-ci-azml'}

In [0]:
workspace = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.


In [0]:
# flake8: noqa

! pip install scikit-image
! pip install azure-storage-blob
! pip install cgmzscore==3.0.3
! pip install azure-storage-blob
! pip install bunch
! pip install tensorflow==2.5.0
! pip install tqdm
! pip install glob2

     |████████▎       

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 17.7 MB 6.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Not uninstalling numpy at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f94c9aea-fdb5-4bdc-b16a-b769beb83275
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Not uninstalling scipy at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f94c9aea-fdb5-4bdc-b16a-b769beb83275
    Can't uninstall 'scipy'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-dataset-runtime 1.52.0 requires numpy!=

In [0]:
from datetime import datetime, timezone
from multiprocessing.dummy import Pool as ThreadPool
import os
from pathlib import Path
from typing import Dict, List, Tuple
# import subprocess as sp
import json
import numpy as np
# from distutils.dir_util import copy_tree

import pandas as pd
import psycopg2
from azure.storage.blob import BlobServiceClient

# from PIL import Image
from cgmml.common.data_utilities.mlpipeline_utils import ArtifactProcessor
from cgmml.common.data_utilities.rgbd_matching import match_df_with_depth_and_image_artifacts
from cgmzscore.src.main import z_score_wfh, z_score_lhfa
from cgmml.common.depthmap_toolkit.depthmap import Depthmap
# from cgmml.common.background_segmentation.deeplab.deeplab_model import get_deeplab_model, PERSON_SEGMENTATION


In [0]:
# Constants
ENV_PROD = "env_prod"
ENV_QA = "env_qa"
ENV_SANDBOX = "env_dev"


In [0]:
# Configuration
# ENV_PROD = "sample_env"
ENV = ENV_PROD 

DBFS_DIR = f"/tmp/{ENV}"

DATASET_TYPE = 'depthmap'  # Supported: 'rgbd' and 'depthmap'

DATA_CATEGORY = 'Test'  # Supported: 'Train' and 'Test'

In [0]:
SECRET_SCOPE = "cgm-ml-scope"

In [0]:
host = dbutils.secrets.get(scope=SECRET_SCOPE, key="mlapi-db-host")
user = dbutils.secrets.get(scope=SECRET_SCOPE, key="mlapi-db-user")
password = dbutils.secrets.get(scope=SECRET_SCOPE, key="mlapi-db-pw")
conn = psycopg2.connect(host=host, database='cgm-ml', user=user, password=password)

In [0]:
sql_cursor = conn.cursor()

In [0]:
SQL_QUERY_BASE = f"""
    SELECT f.file_path, f.created as timestamp,
           s.id as scan_id, s.scan_type_id as scan_step, s.version as scan_version,
           m.height, m.weight, m.muac,
           a.ord as order_number, a.format,
           a.file_id as artifact,
           a.id as artifact_id,
           di.model as device_model,
           dc.description,
           p.id as person_id, p.age, p.sex
    FROM file f
    INNER JOIN artifact a ON f.id = a.file_id
    INNER JOIN scan s     ON s.id = a.scan_id
    INNER JOIN measure m  ON m.person_id = s.person_id
    INNER JOIN person p ON p.id = s.person_id
    INNER JOIN child_data_category cdc ON p.id = cdc.person_id
    INNER JOIN data_category dc ON dc.id = cdc.data_category_id
    INNER JOIN device_info di ON di.id = s.device_info_id
    WHERE dc.description = '{DATA_CATEGORY}'
    AND di.model = 'HUAWEI VOG-L29'"""

if DATASET_TYPE == 'depthmap':
    SQL_QUERY = f"""{SQL_QUERY_BASE}
    AND a.format IN ('depth', 'application/zip');"""
elif DATASET_TYPE == 'rgbd':
    SQL_QUERY = f"""{SQL_QUERY_BASE}
    AND a.ord IS NOT NULL;"""
elif DATASET_TYPE =='rgb':
    SQL_QUERY = f"""{SQL_QUERY_BASE}
    AND a.format IN('rgb');"""
else:
  raise NameError(f'Unknown dataset type: {DATASET_TYPE}')

sql_cursor.execute(SQL_QUERY)

# Get multiple query_result rows
NUM_ARTIFACTS = None
query_results_tmp: List[Tuple[str]] = sql_cursor.fetchall() if NUM_ARTIFACTS is None else sql_cursor.fetchmany(NUM_ARTIFACTS)

In [0]:
_column_names: Tuple[psycopg2.extensions.Column] = sql_cursor.description
df = pd.DataFrame(query_results_tmp, columns=list(map(lambda x: x.name, _column_names)))

def clean_timestamp(timestamp_str: str):
    return timestamp_str.replace(' ', '-').replace(':', '-'). replace('.', '-')
df['timestamp'] = df['timestamp'].astype(str).apply(clean_timestamp)
print("Total number of unique scans in fetched:", len(pd.unique(df['scan_id'])))
print("Total number of unique artifacts fetched:",len(pd.unique(df['file_path'])))

Total number of unique scans in fetched: 0
Total number of unique artifacts fetched: 0


In [0]:
df = df.loc[(df['age']>=365/2) & (df['age']<=365*5)]
print("Shape within age range:", df.shape)
print("Total number of unique scans in fetched:", len(pd.unique(df['scan_id'])))
print("Total number of unique artifacts fetched:",len(pd.unique(df['file_path'])))
df.head()

Shape within age range: (0, 17)
Total number of unique scans in fetched: 0
Total number of unique artifacts fetched: 0


,file_path,timestamp,scan_id,scan_step,scan_version,height,weight,muac,order_number,format,artifact,artifact_id,device_model,description,person_id,age,sex


# separate RGB and depthmap

In [0]:
df_rgb = df[df['format']=='rgb']
df_depth = df[df['format']=='depth']
df_rgb.rename(columns={'file_path': 'rgb_file_path', 'artifact_id': 'rgb_artifact_id'},inplace=True)
df_rgb

/databricks/python/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 return super().rename(
Out[32]:

,rgb_file_path,timestamp,scan_id,scan_step,scan_version,height,weight,muac,order_number,format,artifact,rgb_artifact_id,device_model,description,person_id,age,sex
17,1678302452280/750230e0-81d2-48e2-a253-8cc38754...,2023-03-08-19-07-32-306173,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,1,rgb,7a7478c0-bde4-11ed-8a41-a3165d89ce9d,aaba90a9-bde4-11ed-8a41-b76e168ac8c5,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male
18,1678302452442/2dfc4d8d-9e41-4ef6-8799-4a29d2ca...,2023-03-08-19-07-32-462386,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,2,rgb,7a8ce2c0-bde4-11ed-8a41-c3eaeabdc8e1,aaba90aa-bde4-11ed-8a41-2bbf7d35cf56,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male
20,1678302452596/324349ae-2dc5-4cf2-ab34-f4371184...,2023-03-08-19-07-32-618706,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,3,rgb,7aa3c620-bde4-11ed-8a41-af52cb18d6d4,aabc1740-bde4-11ed-8a41-1f392f230213,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male
21,1678302452723/e4c6a34b-e4e2-4e37-85b2-7d97db9a...,2023-03-08-19-07-32-743681,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,4,rgb,7ab922e0-bde4-11ed-8a41-27905f4dc2bb,aabc1741-bde4-11ed-8a41-ffc95aa436e1,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male
25,1678302452856/084d62b2-1f2f-4103-b4af-2b344c56...,2023-03-08-19-07-32-868716,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,5,rgb,7ac9ebc0-bde4-11ed-8a41-c78127472290,aabc1742-bde4-11ed-8a41-87a7849f8506,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313170,1678567056802/3e2ee306-7915-4988-8b0d-d425d013...,2023-03-11-20-37-36-832155,a26aa920-c04c-11ed-8a41-b3771b72f02a,200,v1.2.1,68.2,8.30,14.3,7,rgb,8f0b8de0-c04c-11ed-8a41-47462d36fbee,a26f3d05-c04c-11ed-8a41-13ddbe0fad38,HUAWEI VOG-L29,Test,9e87c9a0-c04c-11ed-8a41-bfa81c1606d8,398,male
313171,1678567056802/3e2ee306-7915-4988-8b0d-d425d013...,2023-03-11-20-37-36-832155,a26aa920-c04c-11ed-8a41-b3771b72f02a,200,v1.2.1,68.2,8.30,14.3,7,rgb,8f0b8de0-c04c-11ed-8a41-47462d36fbee,a26f3d05-c04c-11ed-8a41-13ddbe0fad38,HUAWEI VOG-L29,Test,9e87c9a0-c04c-11ed-8a41-bfa81c1606d8,398,male
313172,1678567079330/e169d74d-9de8-4a20-9708-28d03762...,2023-03-11-20-37-59-363709,a3328d00-c04c-11ed-8a41-23fb75cf9ed3,202,v1.2.1,68.2,8.30,14.3,8,rgb,9c795c00-c04c-11ed-8a41-335cb92ae619,a33720e6-c04c-11ed-8a41-af635c3ae03e,HUAWEI VOG-L29,Test,9e87c9a0-c04c-11ed-8a41-bfa81c1606d8,398,male
313173,1678567079505/96cb14e3-e5da-40c8-a0a9-8cf9a89c...,2023-03-11-20-37-59-535505,a3328d00-c04c-11ed-8a41-23fb75cf9ed3,202,v1.2.1,68.2,8.30,14.3,9,rgb,9c934ca0-c04c-11ed-8a41-232afc9fdb85,a33720e7-c04c-11ed-8a41-6b79bb263f93,HUAWEI VOG-L29,Test,9e87c9a0-c04c-11ed-8a41-bfa81c1606d8,398,male


In [0]:
def calculate_lhfa_zscore(rows):
    sex = rows['sex']
    if sex == 'female':
        gender = 'F'
    else:
        gender = 'M'
    age = rows['age']
    height = rows['height']
    score = z_score_lhfa(age_in_days=age, sex=gender, height=height)
    return score

In [0]:
def diagnosis_on_lhfa(rows):
    lhfa = rows['zscore_lhfa']
    if lhfa < -3:
        class_lhfa = 'Severely Stunted'
    elif -3 <= lhfa < -2:
        class_lhfa = 'Moderately Stunted'
    else:
        class_lhfa = 'Not Stunted'
    return class_lhfa

In [0]:
dataframe = results.drop_duplicates(subset=['scan_id'])
dataframe['zscore_lhfa'] = dataframe.apply(calculate_lhfa_zscore,axis =1)
dataframe['diagnosis_lhfa'] = dataframe.apply(diagnosis_on_lhfa,axis=1)
dataframe.head()

<command-2993228891058628>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['zscore_lhfa'] = dataframe.apply(calculate_lhfa_zscore,axis =1)
<command-2993228891058628>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['diagnosis_lhfa'] = dataframe.apply(diagnosis_on_lhfa,axis=1)


,scan_id,age,sex,scan_step,depth_artifact_id,person_id,height,prediction_height,file_path,timestamp,scan_version,weight,muac,order_number,format,artifact,artifact_id,device_model,description,pose_score,pose_workflow,pose_results,ord,no_of_person,zscore_lhfa,diagnosis_lhfa
0,9929ccc0-bde4-11ed-8a41-831d357776db,837,female,102,9929ccc4-bde4-11ed-8a41-fbc4fa001cdc,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,83.45982,1678302477500/aafa03cf-ef2f-4749-b635-6a934226...,2023-03-08-19-07-57-509687,v1.2.0,11.10,14.4,1,depth,8979afc0-bde4-11ed-8a41-e3f44c8ddfdb,9929ccc4-bde4-11ed-8a41-fbc4fa001cdc,HUAWEI VOG-L29,Test,0.848910,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(391.114, 59.477055), (7...",1,1,-0.79,Not Stunted
1,986c9740-bde4-11ed-8a41-a7f3907ad526,837,female,101,986fa48c-bde4-11ed-8a41-6ba2ac65bb83,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,83.49455,1678302452120/af16a644-271e-46ac-abcd-3c3973e3...,2023-03-08-19-07-32-134324,v1.2.0,11.10,14.4,15,depth,7a5a8820-bde4-11ed-8a41-b3378eb81a2f,986fa48c-bde4-11ed-8a41-6ba2ac65bb83,HUAWEI VOG-L29,Test,0.760279,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(320.6511, 128.28441), (...",15,1,-0.79,Not Stunted
6,99c26340-bde4-11ed-8a41-27adee4942f4,837,female,100,99c26345-bde4-11ed-8a41-4383c3d62fc5,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,84.19260,1678302501138/1356bd9b-6504-48e9-905c-4a356da9...,2023-03-08-19-08-21-150713,v1.2.0,11.10,14.4,5,depth,979263e0-bde4-11ed-8a41-ab18a162463d,99c26345-bde4-11ed-8a41-4383c3d62fc5,HUAWEI VOG-L29,Test,0.959110,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(395.20273, 86.61575), (...",5,1,-0.79,Not Stunted
20,aab78361-bde4-11ed-8a41-9382d32711a5,1088,male,100,aaba90a0-bde4-11ed-8a41-c712d18e14f5,aab5fcc0-bde4-11ed-8a41-2bb725dac827,86.9,87.75088,1678302451222/e481e681-a9dc-4501-86aa-c1a06ed9...,2023-03-08-19-07-31-228055,v1.2.0,11.35,15.5,1,depth,79cfad40-bde4-11ed-8a41-17fe789699ea,aaba90a0-bde4-11ed-8a41-c712d18e14f5,HUAWEI VOG-L29,Test,0.947321,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(310.09265, 137.10991), ...",1,1,-2.44,Moderately Stunted
42,abc28ac0-bde4-11ed-8a41-dfa33b2f2859,1088,male,101,abca2be5-bde4-11ed-8a41-07ff37142b41,aab5fcc0-bde4-11ed-8a41-2bb725dac827,86.9,86.35381,1678302530159/989efd21-904b-4650-b1a8-0175c7bc...,2023-03-08-19-08-50-166896,v1.2.0,11.35,15.5,25,depth,a8dcf700-bde4-11ed-8a41-6bf0c2a71f58,abca2be5-bde4-11ed-8a41-07ff37142b41,HUAWEI VOG-L29,Test,0.749408,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(255.64192, 69.648865), ...",25,1,-2.44,Moderately Stunted


In [0]:
all_data = results.merge(dataframe, on='scan_id', how='inner',suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
print("Total number of unique scans in fetched:", len(pd.unique(all_data['scan_id'])))
print("Total number of unique artifacts fetched:",len(pd.unique(all_data['file_path'])))

Total number of unique scans in fetched: 13136
Total number of unique artifacts fetched: 189611


In [0]:
CONNECTION_STR_SRC_SA = dbutils.secrets.get(scope=SECRET_SCOPE, key="mlapi-sa-sas-token")
# STORAGE_ACCOUNT_NAME = dbutils.secrets.get(scope=SECRET_SCOPE, key="mlapi-sa-name")
CONTAINER_NAME_SRC_SA = "cgm-result"

# COMMAND ----------

BLOB_SERVICE_CLIENT = BlobServiceClient.from_connection_string(CONNECTION_STR_SRC_SA)

# COMMAND ----------

def download_from_blob_storage(src: str, dest: str, container: str):
    blob_client = BLOB_SERVICE_CLIENT.get_blob_client(container=container, blob=src)
    Path(dest).parent.mkdir(parents=True, exist_ok=True)
    with open(dest, "wb") as download_file:
        content = blob_client.download_blob().readall()
        download_file.write(content)

# COMMAND ----------

# Gather file_paths
_file_paths = df['file_path'].tolist()

# Remove duplicates
_file_paths = list(set(_file_paths))
print(f"Number of files to download: {len(_file_paths)}")

# COMMAND ----------

# Download
def _download(full_name):
    download_from_blob_storage(src=full_name, dest=f"/dbfs{DBFS_DIR}/{full_name}", container=CONTAINER_NAME_SRC_SA)

# COMMAND ----------

NUM_THREADS = 32
pool = ThreadPool(NUM_THREADS)
results = pool.map(_download, _file_paths)


Number of files to download: 254278

In [0]:
from typing import Dict, List, Tuple
SQL_QUERY_BASE_POSE = f""" select ar.artifact_id, r.data->>'Pose Scores'  as pose_score, r.result_workflow_id as pose_Workflow,r.data->>'Pose Results' as pose_results, r.scan_id , a.ord, a.format from artifact_result ar 
INNER JOIN "result" r on r.id = ar.result_id 
INNER JOIN artifact a ON a.id =ar.artifact_id 
and r.result_workflow_id = '{POSE_MODEL}'  and r.data ? 'Pose Scores' and r.data ? 'Pose Results'"""

NUM_ARTIFACTS= None
sql_cursor.execute(SQL_QUERY_BASE_POSE)
query_results_tmp_pose: List[Tuple[str]] = sql_cursor.fetchall() if NUM_ARTIFACTS is None else sql_cursor.fetchmany(NUM_ARTIFACTS)
_column_names: Tuple[psycopg2.extensions.Column] = sql_cursor.description
# df_pose = pd.DataFrame(query_results_tmp_pose, columns=list(map(lambda x: x.name, _column_names)))

In [0]:
SQL_QUERY_BASE_POSE = f"""select ar.artifact_id, r.data->>'no of person using pose' as no_of_person, r.scan_id from artifact_result ar join "result" r on r.id = ar.result_id and r.result_workflow_id = '{POSE_MODEL}' and r.data ? 'no of person using pose'"""

sql_cursor.execute(SQL_QUERY_BASE_POSE)
query_results_tmp_pose: List[Tuple[str]] = sql_cursor.fetchall() if NUM_ARTIFACTS is None else sql_cursor.fetchmany(NUM_ARTIFACTS)
_column_names: Tuple[psycopg2.extensions.Column] = sql_cursor.description
df_no_of_person = pd.DataFrame(query_results_tmp_pose, columns=list(map(lambda x: x.name, _column_names)))

In [0]:
total_pose_score = len(pd.unique(df_pose['scan_id']))
total_no_of_person = len(pd.unique(df_no_of_person['scan_id']))
                       
print("Total unique scans for no. of person:", total_no_of_person)
print("Total unqiue scans having pose score:", total_pose_score)

In [0]:
df_all_pose= df_pose.merge(df_no_of_person, on='scan_id', how='inner',suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
df_all_pose

Out[22]:

,artifact_id,pose_score,pose_workflow,pose_results,scan_id,ord,format,no_of_person
0,707c0c81-befb-11ed-8a41-27f2909c9a51,0.84966815,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(303.01175, 356.24164), ...",7078ff41-befb-11ed-8a41-53f4813a7759,1,depth,1
1,707c0c81-befb-11ed-8a41-27f2909c9a51,0.84966815,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(303.01175, 356.24164), ...",7078ff41-befb-11ed-8a41-53f4813a7759,1,depth,1
2,707c0c81-befb-11ed-8a41-27f2909c9a51,0.84966815,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(303.01175, 356.24164), ...",7078ff41-befb-11ed-8a41-53f4813a7759,1,depth,1
3,707c0c81-befb-11ed-8a41-27f2909c9a51,0.84966815,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(303.01175, 356.24164), ...",7078ff41-befb-11ed-8a41-53f4813a7759,1,depth,1
4,707c0c81-befb-11ed-8a41-27f2909c9a51,0.84966815,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(303.01175, 356.24164), ...",7078ff41-befb-11ed-8a41-53f4813a7759,1,depth,1
...,...,...,...,...,...,...,...,...
23303801,6e1b3923-befb-11ed-8a41-77cfd4f0a440,0.9208183,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(317.9896, 295.48273), (...",6e16a540-befb-11ed-8a41-b3f46ff3600f,8,depth,1
23303802,6e1b3923-befb-11ed-8a41-77cfd4f0a440,0.9208183,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(317.9896, 295.48273), (...",6e16a540-befb-11ed-8a41-b3f46ff3600f,8,depth,1
23303803,6e1b3923-befb-11ed-8a41-77cfd4f0a440,0.9208183,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(317.9896, 295.48273), (...",6e16a540-befb-11ed-8a41-b3f46ff3600f,8,depth,1
23303804,6e1b3923-befb-11ed-8a41-77cfd4f0a440,0.9208183,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(317.9896, 295.48273), (...",6e16a540-befb-11ed-8a41-b3f46ff3600f,8,depth,1


In [0]:
pose_dataframe = pd.merge(df, df_all_pose_depth,  how='inner', left_on=['scan_id','order_number'], right_on = ['scan_id','ord'],suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
pose_dataframe

Out[35]:

,file_path,timestamp,scan_id,scan_step,scan_version,height,weight,muac,order_number,format,artifact,artifact_id,device_model,description,person_id,age,sex,pose_score,pose_workflow,pose_results,ord,no_of_person
0,1678302451222/e481e681-a9dc-4501-86aa-c1a06ed9...,2023-03-08-19-07-31-228055,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,1,depth,79cfad40-bde4-11ed-8a41-17fe789699ea,aaba90a0-bde4-11ed-8a41-c712d18e14f5,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male,0.94732094,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(310.09265, 137.10991), ...",1,1
1,1678302450534/97c0ec99-9215-41f2-9505-4d588a8b...,2023-03-08-19-07-30-587402,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,1,depth,796e0540-bde4-11ed-8a41-872d073e7c28,986c9741-bde4-11ed-8a41-3f9a25eba39c,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.7850746,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(392.55573, 265.76773), ...",1,1
2,1678302450741/ae371d42-a801-4e66-b1a4-6518dec4...,2023-03-08-19-07-30-743654,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,2,depth,7987f5e0-bde4-11ed-8a41-7ff6b558dac2,986c9742-bde4-11ed-8a41-1766f3b65f09,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.80245167,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(407.20746, 197.38132), ...",2,1
3,1678302450869/aa3f94a6-ddf2-4156-8cb1-8b885da1...,2023-03-08-19-07-30-884277,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,3,depth,799bcc00-bde4-11ed-8a41-c7f539be7c7e,986fa480-bde4-11ed-8a41-0380d639a4ac,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.86261034,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(422.52188, 211.5413), (...",3,1
4,1678302450975/77d10546-01f4-4a44-aa4c-8326d446...,2023-03-08-19-07-30-978051,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,4,depth,79a987a0-bde4-11ed-8a41-672f32b3e4b1,986fa481-bde4-11ed-8a41-33520e5ae941,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.83610004,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(380.5455, 162.7933), (7...",4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236652,1678304227609/d715800c-1954-4d3b-a4e7-cc3fd0d0...,2023-03-08-19-37-07-618123,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,19,depth,9c9f47a0-bde8-11ed-8a41-df4db33cb7b4,c3438d87-bde8-11ed-8a41-b7f611228c70,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.78645945,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(317.36475, 54.674892), ...",19,1
236653,1678304227730/adab788c-0981-487e-b12f-d86ad03e...,2023-03-08-19-37-07-743072,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,20,depth,9cb31dc0-bde8-11ed-8a41-57932c9cf8a8,c3438d88-bde8-11ed-8a41-7f8c2409b769,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.7973419,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(173.02592, 72.598976), ...",20,1
236654,1678304227872/b1bf2f06-1b04-4fab-98f7-4e2cbbb5...,2023-03-08-19-37-07-883754,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,21,depth,9cc87a80-bde8-11ed-8a41-5786498308af,c3438d89-bde8-11ed-8a41-7f19e2fa74a1,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.7949422,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(212.54254, 71.24314), (...",21,1
236655,1678304227988/02b4352e-a30d-4bb5-a667-c469c444...,2023-03-08-19-37-07-993129,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,22,depth,9cd94360-bde8-11ed-8a41-070929108560,c3438d8a-bde8-11ed-8a41-2bfd3b1a0c25,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.7954041,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(239.69826, 70.736946), ...",22,1


In [0]:
pose_dataframe.drop_duplicates(subset='artifact_id', keep='first', inplace=True)
pose_dataframe

Out[38]:

,file_path,timestamp,scan_id,scan_step,scan_version,height,weight,muac,order_number,format,artifact,artifact_id,device_model,description,person_id,age,sex,pose_score,pose_workflow,pose_results,ord,no_of_person
0,1678302451222/e481e681-a9dc-4501-86aa-c1a06ed9...,2023-03-08-19-07-31-228055,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,1,depth,79cfad40-bde4-11ed-8a41-17fe789699ea,aaba90a0-bde4-11ed-8a41-c712d18e14f5,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male,0.94732094,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(310.09265, 137.10991), ...",1,1
1,1678302450534/97c0ec99-9215-41f2-9505-4d588a8b...,2023-03-08-19-07-30-587402,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,1,depth,796e0540-bde4-11ed-8a41-872d073e7c28,986c9741-bde4-11ed-8a41-3f9a25eba39c,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.7850746,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(392.55573, 265.76773), ...",1,1
2,1678302450741/ae371d42-a801-4e66-b1a4-6518dec4...,2023-03-08-19-07-30-743654,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,2,depth,7987f5e0-bde4-11ed-8a41-7ff6b558dac2,986c9742-bde4-11ed-8a41-1766f3b65f09,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.80245167,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(407.20746, 197.38132), ...",2,1
3,1678302450869/aa3f94a6-ddf2-4156-8cb1-8b885da1...,2023-03-08-19-07-30-884277,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,3,depth,799bcc00-bde4-11ed-8a41-c7f539be7c7e,986fa480-bde4-11ed-8a41-0380d639a4ac,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.86261034,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(422.52188, 211.5413), (...",3,1
4,1678302450975/77d10546-01f4-4a44-aa4c-8326d446...,2023-03-08-19-07-30-978051,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,4,depth,79a987a0-bde4-11ed-8a41-672f32b3e4b1,986fa481-bde4-11ed-8a41-33520e5ae941,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.83610004,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(380.5455, 162.7933), (7...",4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236652,1678304227609/d715800c-1954-4d3b-a4e7-cc3fd0d0...,2023-03-08-19-37-07-618123,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,19,depth,9c9f47a0-bde8-11ed-8a41-df4db33cb7b4,c3438d87-bde8-11ed-8a41-b7f611228c70,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.78645945,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(317.36475, 54.674892), ...",19,1
236653,1678304227730/adab788c-0981-487e-b12f-d86ad03e...,2023-03-08-19-37-07-743072,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,20,depth,9cb31dc0-bde8-11ed-8a41-57932c9cf8a8,c3438d88-bde8-11ed-8a41-7f8c2409b769,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.7973419,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(173.02592, 72.598976), ...",20,1
236654,1678304227872/b1bf2f06-1b04-4fab-98f7-4e2cbbb5...,2023-03-08-19-37-07-883754,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,21,depth,9cc87a80-bde8-11ed-8a41-5786498308af,c3438d89-bde8-11ed-8a41-7f19e2fa74a1,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.7949422,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(212.54254, 71.24314), (...",21,1
236655,1678304227988/02b4352e-a30d-4bb5-a667-c469c444...,2023-03-08-19-37-07-993129,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,201,v1.2.0,72.1,6.90,11.9,22,depth,9cd94360-bde8-11ed-8a41-070929108560,c3438d8a-bde8-11ed-8a41-2bfd3b1a0c25,HUAWEI VOG-L29,Test,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,518,female,0.7954041,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(239.69826, 70.736946), ...",22,1


In [0]:
STANDING_LAYING_MODEL = 'abc6f040-738c-11ec-8cec-1fb69324fccb'
NUM_ARTIFACTS = None
SQL_QUERY_BASE_STANDING = f"""SELECT r.scan_id, r.result_workflow_id,r.id,r.data,ar.artifact_id,a.ord as standing_ord
FROM artifact_result ar 
INNER JOIN "result" r on r.id = ar.result_id 
INNER JOIN artifact a ON a.id =ar.artifact_id 
where r.id=ar.result_id and r.result_workflow_id ='abc6f040-738c-11ec-8cec-1fb69324fccb'""";

# SQL_QUERY_BASE_STANDING = f""" select scan_id, result_workflow_id, data FROM result where result_workflow_id = '{STANDING_LAYING_MODEL}'"""
sql_cursor.execute(SQL_QUERY_BASE_STANDING)
query_results_tmp_standing: List[Tuple[str]] = sql_cursor.fetchall() if NUM_ARTIFACTS is None else sql_cursor.fetchmany(NUM_ARTIFACTS)

_column_names: Tuple[psycopg2.extensions.Column] = sql_cursor.description
df_standing = pd.DataFrame(query_results_tmp_standing, columns=list(map(lambda x: x.name, _column_names)))
print("Total number of unique scans in standing:", len(pd.unique(df_standing['scan_id'])))

Total number of unique scans in standing: 0


In [0]:
df_pose_data = df_pose.merge(df_no_of_person, on='artifact_id', how='inner', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
print("Any null values in pose results:",df_pose_data.isnull().values.any())
df_pose_data.head()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4496430288336214>:1
----> 1 df_pose_data = df_pose.merge(df_no_of_person, on='artifact_id', how='inner', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
      2 print("Any null values in pose results:",df_pose_data.isnull().values.any())
      3 df_pose_data.head()

NameError: name 'df_pose' is not defined

In [0]:
print('All pose data:', len(pd.unique(df_pose_data['scan_id'])))
print('total artifacts in pose data:', len(df_pose_data))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4496430288336215>:1
----> 1 print('All pose data:', len(pd.unique(df_pose_data['scan_id'])))
      2 print('total artifacts in pose data:', len(df_pose_data))

NameError: name 'df_pose_data' is not defined

In [0]:
pose_dataframe = pd.merge(df, df_pose_data,  how='left', left_on=['scan_id','order_number'], right_on = ['scan_id','ord'],suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
pose_dataframe.head()

In [0]:
def standing_score(rows):
  data = rows['data']
  prediction = data['standing']
  score = prediction[1:-1]
  score = float(score)
  return score

In [0]:
# creating a column that have standing score value from the models in integer format
df_standing['standing'] = df_standing.apply(standing_score,axis=1)
df_standing = df_standing.drop(['data'], axis=1)
df_standing.head()

In [0]:
pose_dataframe = pd.merge(pose_dataframe, df_standing,  how='left', left_on=['scan_id','order_number'], right_on = ['scan_id','standing_ord'],suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
pose_dataframe.head()

In [0]:
df = pose_dataframe.drop_duplicates(subset='depth_artifact_id', keep="first")
len(df)

In [0]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('/dbfs/prashant/test_data.csv')
len(df)
df.head()

,file_path,timestamp,scan_id,scan_step,scan_version,height,weight,muac,order_number,format,artifact,artifact_id,device_model,description,person_id,age,sex,pose_score,pose_workflow,pose_results,ord,no_of_person
0,1678302451222/e481e681-a9dc-4501-86aa-c1a06ed9...,2023-03-08-19-07-31-228055,aab78361-bde4-11ed-8a41-9382d32711a5,100,v1.2.0,86.9,11.35,15.5,1,depth,79cfad40-bde4-11ed-8a41-17fe789699ea,aaba90a0-bde4-11ed-8a41-c712d18e14f5,HUAWEI VOG-L29,Test,aab5fcc0-bde4-11ed-8a41-2bb725dac827,1088,male,0.947321,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(310.09265, 137.10991), ...",1,1
1,1678302450534/97c0ec99-9215-41f2-9505-4d588a8b...,2023-03-08-19-07-30-587402,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,1,depth,796e0540-bde4-11ed-8a41-872d073e7c28,986c9741-bde4-11ed-8a41-3f9a25eba39c,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.785075,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(392.55573, 265.76773), ...",1,1
2,1678302450741/ae371d42-a801-4e66-b1a4-6518dec4...,2023-03-08-19-07-30-743654,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,2,depth,7987f5e0-bde4-11ed-8a41-7ff6b558dac2,986c9742-bde4-11ed-8a41-1766f3b65f09,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.802452,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(407.20746, 197.38132), ...",2,1
3,1678302450869/aa3f94a6-ddf2-4156-8cb1-8b885da1...,2023-03-08-19-07-30-884277,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,3,depth,799bcc00-bde4-11ed-8a41-c7f539be7c7e,986fa480-bde4-11ed-8a41-0380d639a4ac,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.862610,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(422.52188, 211.5413), (...",3,1
4,1678302450975/77d10546-01f4-4a44-aa4c-8326d446...,2023-03-08-19-07-30-978051,986c9740-bde4-11ed-8a41-a7f3907ad526,101,v1.2.0,86.0,11.10,14.4,4,depth,79a987a0-bde4-11ed-8a41-672f32b3e4b1,986fa481-bde4-11ed-8a41-33520e5ae941,HUAWEI VOG-L29,Test,986b10a0-bde4-11ed-8a41-bf9971bab2fb,837,female,0.836100,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(380.5455, 162.7933), (7...",4,1


In [0]:
from skimage.transform import resize
from PIL import Image
from matplotlib import pyplot as plt
import tempfile

def read_rgb_data(rgb_fpath, height,width):
  if rgb_fpath:
    pil_im = Image.open(rgb_fpath)
    pil_im = pil_im.rotate(-90, expand=True)
    rgb_height, rgb_width = pil_im.width, pil_im.height  # Weird switch
          # assert rgb_width / width == rgb_height / height, f'{rgb_width} / {width} != {rgb_height} / {height}'
    pil_im = pil_im.resize((rgb_height,rgb_width), Image.ANTIALIAS)
    rgb_array = np.asarray(pil_im)
  else:
    rgb_array = None
  return rgb_array

In [0]:
if DATASET_TYPE == 'rgbd':
    fused_artifacts_dicts = match_df_with_depth_and_image_artifacts(df)
    df_to_process = pd.DataFrame(fused_artifacts_dicts)
else:
    df_to_process = df

In [0]:
query_results_dicts: List[dict] = df_to_process.to_dict('records')

In [0]:
# DBFS_DIR ='sample'
DATASET_TYPE = 'depthmap'
input_dir = f"/dbfs/depth{DBFS_DIR}"
output_dir = f"/dbfs/test/{DBFS_DIR}"
artifact_processor = ArtifactProcessor(input_dir, output_dir, dataset_type=DATASET_TYPE, should_rotate_rgb=False)

In [0]:
RGB_HEIGHT = 1440
RGB_WEIGHT = 1080

In [0]:
import pickle
def process_rgb(artifact_dict,input_dir,output_dir):
  zip_input_full_path = f"{input_dir}/{artifact_dict['file_path']}"
  layers = read_rgb_data(zip_input_full_path,RGB_HEIGHT,RGB_WEIGHT)
  timestamp = artifact_dict['timestamp']
  scan_id = artifact_dict['scan_id']
  scan_step = artifact_dict['scan_step']
  order_number = artifact_dict['order_number']
  person_id = artifact_dict['person_id']
  pickle_output_path = f"scans/{person_id}/{scan_step}/pc_{scan_id}_{timestamp}_{scan_step}_{order_number}.p"
  target_dict = {**artifact_dict}
  # Write into pickle
  pickle_output_full_path = f"{output_dir}/{pickle_output_path}"
  Path(pickle_output_full_path).parent.mkdir(parents=True, exist_ok=True)
  pickle.dump((layers, target_dict), open(pickle_output_full_path, "wb"))
  return pickle_output_full_path


In [0]:
## Function to process RGB
def map_fct_rgb(query_result_dict):
    return query_result_dict, process_rgb(query_result_dict,input_dir,output_dir)

In [0]:
## Function to depthmap
def map_fct(query_result_dict):
#     print("the_query_dict", query_result_dict)
    return query_result_dict, artifact_processor.create_and_save_pickle(query_result_dict)

In [0]:
# Processing all artifacts at once
rdd = spark.sparkContext.parallelize(query_results_dicts,48)
print(rdd.getNumPartitions())
rdd_processed = rdd.map(map_fct)
processed_dicts_and_fnames = rdd_processed.collect()
print(processed_dicts_and_fnames[:3])

48


In [0]:
import pickle
with open('/dbfs/prashant/angle.pkl', 'rb') as f:
    processed_dicts_and_fnames = pickle.load(f)
len(processed_dicts_and_fnames)

Out[3]: 189611

In [0]:
processed_dicts_and_fnames[:2]

Out[5]: [({'scan_id': '9929ccc0-bde4-11ed-8a41-831d357776db',
   'age': 837,
   'sex': 'female',
   'scan_step': 102,
   'depth_artifact_id': '9929ccc4-bde4-11ed-8a41-fbc4fa001cdc',
   'person_id': '986b10a0-bde4-11ed-8a41-bf9971bab2fb',
   'height': 86.0,
   'prediction_height': 83.45982,
   'file_path': '1678302477500/aafa03cf-ef2f-4749-b635-6a934226c7fc',
   'timestamp': '2023-03-08-19-07-57-509687',
   'scan_version': 'v1.2.0',
   'weight': 11.1,
   'muac': 14.4,
   'order_number': 1,
   'format': 'depth',
   'artifact': '8979afc0-bde4-11ed-8a41-e3f44c8ddfdb',
   'artifact_id': '9929ccc4-bde4-11ed-8a41-fbc4fa001cdc',
   'device_model': 'HUAWEI VOG-L29',
   'description': 'Test',
   'pose_score': 0.8489104,
   'pose_workflow': 'b04da320-42b3-11ec-9e50-0b91a7b6eecf',
   'pose_results': "{'bbox_coordinates': [(391.114, 59.477055), (765.45825, 1183.8585)], 'bbox_confidence_score': [0.9986998], 'key_points_coordinate': [{'nose': {'x': 288.59903, 'y': 417.53168}}, {'left_eye': {'x': 281.

## Load Model

In [0]:
import tensorflow  as tf
from tensorflow.keras.models import load_model

In [0]:
from azureml.core import Experiment, Run
workspace = Workspace.from_config()
experiment = Experiment(workspace=workspace, name="q4-depthmap-plaincnn-height")
RUN_ID='q4-depthmap-plaincnn-height_1669110716_ae0979c7'
run = Run(experiment, run_id=RUN_ID)
run.download_files(output_paths='.')

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2054704264018394>:1
----> 1 from azureml.core import Experiment, Run
      2 workspace = Workspace.from_config()
      3 experiment = Experiment(workspace=workspace, name="q4-depthmap-plaincnn-height")

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._nest_level == 1
    174     # `level` represents the number of leading dots in a relative imp

In [0]:
MODEL_PATH = 'outputs/best_model.ckpt'
model = load_model(MODEL_PATH,compile=False)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 240, 180, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 180, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 90, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 90, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 120, 90, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 45, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 45, 128)       7

In [0]:
def preprocess_depthmap(depthmap):
    return depthmap.astype("float32")

In [0]:
IMAGE_TARGET_HEIGHT= int(240)
IMAGE_TARGET_WIDTH= int(180)
NORMALIZATION_VALUE=7.5
TARGET_NAMES=['height']

In [0]:
def preprocess_targets(targets: dict, target_names: List[str]) -> np.ndarray:
    if target_names is not None:
        targets = [targets[target_name] for target_name in target_names]
    return np.array(targets).astype("float32")

In [0]:
debug = []
all_files = []
def preprocess(path):
  all_files.append(path)
  exists = os.path.isfile(path)
  if not exists:
    return "File not found"
  try:
    depthmap, targets = pickle.load(open(path, "rb"))
    depthmap = preprocess_depthmap(depthmap)
    depthmap = depthmap / NORMALIZATION_VALUE
  except:
    print(" not processed for preprocess:", path)
  depthmap = tf.image.resize(depthmap, (IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH))
  targets = preprocess_targets(targets, TARGET_NAMES)
  depthmap.set_shape((IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH, 1))
  return depthmap, targets


In [0]:
def predict(depthmap):
  sample = np.expand_dims(depthmap, axis=0)
  inference = model.predict(sample)
  return inference

In [0]:
result_file = []
def _process_result(files_list):
  metadata = files_list[0]
  if metadata == '':
    return 
  pickle_file = files_list[1]
  try:
    depthmap, target = preprocess(pickle_file)
    prediction = predict(depthmap)
    result_file.append([metadata['scan_id'],metadata['age'],metadata['sex'],metadata['scan_step'],metadata['artifact_id'],metadata['person_id'],target[0],prediction[0][0]])
  except ValueError:
    print('value error')
  
  # result_file.append([metadata['scan_id'],metadata['age'],metadata['sex'],metadata['zscore_lhfa'],metadata['diagnosis_lhfa'],metadata['scan_step'],metadata['artifact_id'],metadata['person_id'],target[0],prediction[0][0]])
  

In [0]:
#sample prediction
files = sample[1]
depthmap, target = preprocess(files)
image = np.expand_dims(depthmap, axis=0)
plt.imshow(depthmap)
inference = model.predict(image)
print(inference)
print(target)
# depthmap, target = preprocess(pickle_file)
# prediction = predict(depthmap)

In [0]:
# NUM_THREADS = 60
# pool = ThreadPool(NUM_THREADS)
# all_results = pool.map(_process_result, processed_dicts_and_fnames)
# result_file 
from tqdm import tqdm
NUM_THREADS = 42
pool = ThreadPool(NUM_THREADS)

# Define a wrapper function for _process_result that returns True after the result is processed
def process_result_wrapper(files):
    result = _process_result(files)
    return result, True

# Use tqdm to display a progress bar while processing the results
all_results = []
for result, _ in tqdm(pool.imap_unordered(process_result_wrapper, data2), total=len(data2)):
    all_results.append(result)


  3%|▎         | 3042/94922 [01:51<47:52, 31.99it/s]

value error
value error


  4%|▍         | 3602/94922 [02:12<58:13, 26.14it/s]

value error


  4%|▍         | 4043/94922 [02:28<1:09:00, 21.95it/s]

...

 98%|█████████▊| 92744/94922 [58:47<01:31, 23.75it/s]

value error


 98%|█████████▊| 93290/94922 [59:07<00:56, 28.72it/s]

value errorvalue error



100%|██████████| 94922/94922 [1:00:43<00:00, 26.05it/s]


In [0]:
result_data = pd.DataFrame(result_file,columns = ['scan_id','age','sex','scan_step','depth_artifact_id','person_id','height','prediction_height'])
result_data.head()


,scan_id,age,sex,scan_step,depth_artifact_id,person_id,height,prediction_height
0,8f02ba20-bf21-11ed-8a41-eb2828c5f0cc,1587,male,100,8f0440c0-bf21-11ed-8a41-9b51ad1c6d58,8e564d80-bf21-11ed-8a41-271aaf27a37d,97.699997,97.987198
1,8f02ba20-bf21-11ed-8a41-eb2828c5f0cc,1587,male,100,8f0440c1-bf21-11ed-8a41-1fcb836edfea,8e564d80-bf21-11ed-8a41-271aaf27a37d,97.699997,97.601074
2,8fb9d520-bf21-11ed-8a41-ebe6292f83f9,1587,male,101,8fbe6901-bf21-11ed-8a41-ab004d3503ab,8e564d80-bf21-11ed-8a41-271aaf27a37d,97.699997,98.910271
3,8fb9d520-bf21-11ed-8a41-ebe6292f83f9,1587,male,101,8fc17643-bf21-11ed-8a41-eb95321a001d,8e564d80-bf21-11ed-8a41-271aaf27a37d,97.699997,98.015739
4,8fb9d520-bf21-11ed-8a41-ebe6292f83f9,1587,male,101,8fbb5bc7-bf21-11ed-8a41-0fa46a458f43,8e564d80-bf21-11ed-8a41-271aaf27a37d,97.699997,97.828529


In [0]:
results = pd.merge(result_data,df, how='left', left_on=['depth_artifact_id'], right_on = ['artifact_id'],suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
results

,scan_id,age,sex,scan_step,depth_artifact_id,person_id,height,prediction_height,file_path,timestamp,scan_version,weight,muac,order_number,format,artifact,artifact_id,device_model,description,pose_score,pose_workflow,pose_results,ord,no_of_person
0,9929ccc0-bde4-11ed-8a41-831d357776db,837,female,102,9929ccc4-bde4-11ed-8a41-fbc4fa001cdc,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,83.459820,1678302477500/aafa03cf-ef2f-4749-b635-6a934226...,2023-03-08-19-07-57-509687,v1.2.0,11.10,14.4,1,depth,8979afc0-bde4-11ed-8a41-e3f44c8ddfdb,9929ccc4-bde4-11ed-8a41-fbc4fa001cdc,HUAWEI VOG-L29,Test,0.848910,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(391.114, 59.477055), (7...",1,1
1,986c9740-bde4-11ed-8a41-a7f3907ad526,837,female,101,986fa48c-bde4-11ed-8a41-6ba2ac65bb83,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,83.494550,1678302452120/af16a644-271e-46ac-abcd-3c3973e3...,2023-03-08-19-07-32-134324,v1.2.0,11.10,14.4,15,depth,7a5a8820-bde4-11ed-8a41-b3378eb81a2f,986fa48c-bde4-11ed-8a41-6ba2ac65bb83,HUAWEI VOG-L29,Test,0.760279,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(320.6511, 128.28441), (...",15,1
2,986c9740-bde4-11ed-8a41-a7f3907ad526,837,female,101,986c9741-bde4-11ed-8a41-3f9a25eba39c,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,83.080635,1678302450534/97c0ec99-9215-41f2-9505-4d588a8b...,2023-03-08-19-07-30-587402,v1.2.0,11.10,14.4,1,depth,796e0540-bde4-11ed-8a41-872d073e7c28,986c9741-bde4-11ed-8a41-3f9a25eba39c,HUAWEI VOG-L29,Test,0.785075,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(392.55573, 265.76773), ...",1,1
3,9929ccc0-bde4-11ed-8a41-831d357776db,837,female,102,9929ccc1-bde4-11ed-8a41-c72ffd5f6d10,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,84.496620,1678302477187/716155c6-1db2-4577-ad21-073d703d...,2023-03-08-19-07-57-197184,v1.2.0,11.10,14.4,7,depth,894a6260-bde4-11ed-8a41-8b4b2db07fa3,9929ccc1-bde4-11ed-8a41-c72ffd5f6d10,HUAWEI VOG-L29,Test,0.868541,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(443.69577, 120.54117), ...",7,1
4,986c9740-bde4-11ed-8a41-a7f3907ad526,837,female,101,986fa486-bde4-11ed-8a41-c322c0331626,986b10a0-bde4-11ed-8a41-bf9971bab2fb,86.0,84.466150,1678302451518/d4236e86-55ad-4d19-bd4c-46d9f8c6...,2023-03-08-19-07-31-524946,v1.2.0,11.10,14.4,9,depth,79fd7400-bde4-11ed-8a41-f3c8eacde8ee,986fa486-bde4-11ed-8a41-c322c0331626,HUAWEI VOG-L29,Test,0.818789,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(418.20425, 114.70737), ...",9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189606,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,518,female,201,c3438d8a-bde8-11ed-8a41-2bfd3b1a0c25,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,72.1,71.247635,1678304227988/02b4352e-a30d-4bb5-a667-c469c444...,2023-03-08-19-37-07-993129,v1.2.0,6.90,11.9,22,depth,9cd94360-bde8-11ed-8a41-070929108560,c3438d8a-bde8-11ed-8a41-2bfd3b1a0c25,HUAWEI VOG-L29,Test,0.795404,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(239.69826, 70.736946), ...",22,1
189607,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,518,female,201,c3438d89-bde8-11ed-8a41-7f19e2fa74a1,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,72.1,72.103880,1678304227872/b1bf2f06-1b04-4fab-98f7-4e2cbbb5...,2023-03-08-19-37-07-883754,v1.2.0,6.90,11.9,21,depth,9cc87a80-bde8-11ed-8a41-5786498308af,c3438d89-bde8-11ed-8a41-7f19e2fa74a1,HUAWEI VOG-L29,Test,0.794942,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(212.54254, 71.24314), (...",21,1
189608,c33ef9a2-bde8-11ed-8a41-7358f1062fc3,518,female,201,c3438d85-bde8-11ed-8a41-37f88ab66d7d,c33ef9a0-bde8-11ed-8a41-c38eda4ab358,72.1,70.366035,1678304227418/484e5f5d-51c7-43b1-bb97-def2e2f8...,2023-03-08-19-37-07-430665,v1.2.0,6.90,11.9,17,depth,9c83d060-bde8-11ed-8a41-a7b30e572676,c3438d85-bde8-11ed-8a41-37f88ab66d7d,HUAWEI VOG-L29,Test,0.809759,b04da320-42b3-11ec-9e50-0b91a7b6eecf,"{'bbox_coordinates': [(230.20035, 48.86863), (...",17,1
189609,aab78361-bde4-11ed-8a41-9382d32711a5,1088,male,100,aaba90a1-bde4-11ed-8a41-dfd05e2e99d8,aab